<a href="https://colab.research.google.com/github/hsunsmile/ComfyUI_VLM_nodes/blob/main/nightly/comic_diffusion_v2_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

!apt -y update -qq
!apt -y install -qq aria2
!pip install -q numpy==1.26.4
#!pip install -q protobuf==3.20.3 # Trying an older version of protobuf
#!pip install -q torch torchvision torchaudio torchtext torchdata --extra-index-url https://download.pytorch.org/whl/cu118 -U
#!pip install -q xformers triton -U
#!pip install -q mediapipe addict yapf fvcore omegaconf

In [ ]:
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install -q xformers==0.0.16 triton==2.0.0 -U
!pip install -q mediapipe==0.9.1.0 addict yapf fvcore omegaconf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive
!git clone -b v1.2 https://github.com/camenduru/ComfyUI
%cd ComfyUI
!pip install -q -r requirements.txt
!git reset --hard
!git clone -b v1.2 https://github.com/camenduru/comfy_controlnet_preprocessors /content/drive/MyDrive/ComfyUI/custom_nodes/comfy_controlnet_preprocessors
%cd /content/drive/MyDrive/ComfyUI/custom_nodes/comfy_controlnet_preprocessors
!python install.py --no_download_ckpts
%cd /content/drive/MyDrive/ComfyUI

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)

controlnet_dir = "/content/drive/MyDrive/ComfyUI/models/controlnet"
upscale_dir = "/content/drive/MyDrive/ComfyUI/models/upscale_models"
checkpoints_dir = "/content/drive/MyDrive/ComfyUI/models/checkpoints"

os.makedirs(controlnet_dir, exist_ok=True)
os.makedirs(upscale_dir, exist_ok=True)
os.makedirs(checkpoints_dir, exist_ok=True)

controlnet_files = [
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors", "control_v11e_sd15_ip2p_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors", "control_v11e_sd15_shuffle_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors", "control_v11p_sd15_canny_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors", "control_v11f1p_sd15_depth_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors", "control_v11p_sd15_inpaint_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors", "control_v11p_sd15_lineart_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors", "control_v11p_sd15_mlsd_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors", "control_v11p_sd15_normalbae_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors", "control_v11p_sd15_openpose_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors", "control_v11p_sd15_scribble_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors", "control_v11p_sd15_seg_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors", "control_v11p_sd15_softedge_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors", "control_v11p_sd15s2_lineart_anime_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors", "control_v11f1e_sd15_tile_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p_fp16.yaml", "control_v11e_sd15_ip2p_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_shuffle_fp16.yaml", "control_v11e_sd15_shuffle_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml", "control_v11p_sd15_canny_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml", "control_v11f1p_sd15_depth_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml", "control_v11p_sd15_inpaint_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart_fp16.yaml", "control_v11p_sd15_lineart_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml", "control_v11p_sd15_mlsd_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml", "control_v11p_sd15_normalbae_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml", "control_v11p_sd15_openpose_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml", "control_v11p_sd15_scribble_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_seg_fp16.yaml", "control_v11p_sd15_seg_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml", "control_v11p_sd15_softedge_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml", "control_v11p_sd15s2_lineart_anime_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml", "control_v11f1e_sd15_tile_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_style_sd14v1.pth", "t2iadapter_style_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd14v1.pth", "t2iadapter_sketch_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_seg_sd14v1.pth", "t2iadapter_seg_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_openpose_sd14v1.pth", "t2iadapter_openpose_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_keypose_sd14v1.pth", "t2iadapter_keypose_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd14v1.pth", "t2iadapter_depth_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_color_sd14v1.pth", "t2iadapter_color_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd14v1.pth", "t2iadapter_canny_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd15v2.pth", "t2iadapter_canny_sd15v2.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd15v2.pth", "t2iadapter_depth_sd15v2.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd15v2.pth", "t2iadapter_sketch_sd15v2.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_zoedepth_sd15v1.pth", "t2iadapter_zoedepth_sd15v1.pth")
]

upscale_files = [
    ("https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth", "RealESRGAN_x2plus.pth")
]

checkpoint_files = [
    ("https://huggingface.co/ckpt/Comic-Diffusion/resolve/main/comic-diffusion-V2.ckpt", "comic-diffusion-V2.ckpt")
]

for url, filename in controlnet_files:
    filepath = os.path.join(controlnet_dir, filename)
    if not os.path.exists(filepath):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {controlnet_dir} -o {filename}
    else:
        print(f"{filename} already exists, skipping download.")

for url, filename in upscale_files:
    filepath = os.path.join(upscale_dir, filename)
    if not os.path.exists(filepath):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {upscale_dir} -o {filename}
    else:
        print(f"{filename} already exists, skipping download.")

for url, filename in checkpoint_files:
    filepath = os.path.join(checkpoints_dir, filename)
    if not os.path.exists(filepath):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {checkpoints_dir} -o {filename}
    else:
        print(f"{filename} already exists, skipping download.")

!python main.py --dont-print-server